# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example

Michael Reefe

First things first, we need to import the LOKI code. We can do so locally using the `Pkg` module. First we activate the LOKI module, then we instantiate, precompile, and finally import it with the following block of code.

Some aspects of the code may utilize multiprocessing. To take advantage of this, we first must import the `Distributed` package and add parallel CPU processes. Then, our following imports must be encased in an `@everywhere` block to ensure they are loaded onto each CPU process individually.

In [1]:
using Distributed
procs = addprocs(Sys.CPU_THREADS)

4-element Vector{Int64}:
 2
 3
 4
 5

In [2]:
@everywhere begin
    using Pkg
    Pkg.activate(dirname(@__DIR__))
    Pkg.instantiate()
    Pkg.precompile()
    using Loki
end

┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 259. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576


┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 260. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 261. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 262. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 263. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 264. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 265. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/

┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 303. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 304. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 305. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 306. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 307. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/1P1tQ/src/file.jl:576
┌ Warning: thread = 1 warning: only found 2 / 3 columns around data row: 308. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/

  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/JWST_pipeline/NGC_7469_cal`


      From worker 2:	  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/JWST_pipeline/NGC_7469_cal`
      From worker 4:	  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/JWST_pipeline/NGC_7469_cal`
      From worker 5:	  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/JWST_pipeline/NGC_7469_cal`
      From worker 3:	  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/JWST_pipeline/NGC_7469_cal`


### NOTE 1: 

Alternatively, we could have started julia with the command-line argument `--project=/path/to/Loki`. This is my preferred way of starting the code, but this is difficult to demonstrate using a Jupyter notebook. If you choose to start julia this way, the above block is unnecessary since the project is already activated and precompiled. You would just need to add a single line like `@everywhere using Loki`. Additionally, you would have to modify the `addprocs` call with the argument `exeflags="--project=/path/to/Loki"`, which tells the Distributed module that the worker processes should also be started using the Loki project.  

So, all together, in your file (let's call it `example.jl`) you would have:
```julia
using Distributed
procs = addprocs(Sys.CPU_THREADS, exeflags="--project=/path/to/Loki")
@everywhere using Loki
```
And from the terminal you would start the code using:
`julia --project=/path/to/Loki example.jl`

### NOTE 2:

If you plan on running Loki in a High-Performance Computing (HPC) environment, you can also take advantage of julia's built-in distributed package in a very similar way. In fact, if you plan on running on only a single node, then no changes should be necessary at all from the above example. However, if you wish to spread out over multiple nodes, some changes will be required, since by default the above will only add additional processes onto one node. If you cluster uses Slurm, then you can use the `SlurmClusterManager` package like so:

```julia
using Distributed
using SlurmClusterManager
procs = addprocs(SlurmManager(), exeflags="--project=/path/to/Loki")
@everywhere using Loki
```

Then, you would start julia from within an sbatch script specifying how many nodes, tasks, and CPUs you need. I strongly recommend using the `"--project"` flag approach described above when on a cluster, as I have run into strange issues with the other approach causing julia to start thinking pid files are stale and removing them.

Now we want to load in our data. For this example, we'll be using the channel 1 data for NGC 7469, which is located in the same folder as this notebook. Unfortunately the JWST reduced data does not include a redshift, so we must provide the redshift ourselves.  We can use the `from_fits` function to load in the JWST-formatted FITS files, along with the redshift.

In [3]:
# The redshift of the target object: NGC 7469
z = 0.016317
# The semicolon at the end suppresses printing the output Observation object, which is long and not very enlightening
obs = from_fits(["Level3_ch1-long_s3d.fits", "Level3_ch1-medium_s3d.fits", "Level3_ch1-short_s3d.fits"], z);

[ Info: Initializing DataCube struct from Level3_ch1-long_s3d.fits
[ Info: Initializing DataCube struct from Level3_ch1-medium_s3d.fits


[ Info: Initializing DataCube struct from Level3_ch1-short_s3d.fits


Next, we create some variables that we will use later. We will be fitting channel 1 data, and we can take the `name` property from the Observation object we just loaded in to get the name of the target. Here, `run_name` is just a unique identifier that we will use for this run.

In [4]:
channel = 1
nm = replace(obs.name, " " => "_") 
run_name = "$(nm)_ch$(channel)_nuc_aperture"

"NGC_7469_ch1_nuc_aperture"

Before fitting, we want to do some pre-processing on the data. We have three separate sub-channel cubes ("short", "medium", and "long") that we would like to combine into a single full-channel cube. We also want to convert the data to the rest-frame and mask out / interpolate any bad data points.  All of this is achieved in the next block of code.  Importantly, even if you already have full-channel data, you may want to consider using the sub-channel data and combining them using the methodology below. The reasoning for this is that, at least as of early 2023, the MIRI data processing pipeline can sometimes leave large discontinuous jumps in the continuum level between sub-channel and channel boundaries due to small misalignments in the WCS grids.  The `combine_channels!` function attempts to minimize this problem by adjusting the WCS parameters based on centroiding the images, and it also allows for a constrained rescaling factor (<50%) to be applied between channels.

In [5]:
if isfile("$nm.channel$channel.rest_frame.fits")
    # If we've already performed this step in a previous run, just load in the pre-processed data
    obs = from_fits(["$nm.channel$channel.rest_frame.fits"], obs.z);
    
else
    # Convert to rest-frame wavelength vector, and mask out bad spaxels
    correct!(obs)
    
    # Reproject the sub-channels onto the same WCS grid and combine them into one full channel
    # - The [:A1, :B1, :C1] vector gives the names of each channel to concatenate. By default, JWST subchannels are
    #   given labels of "A" for short, "B" for medium, and "C" for long, followed by the channel number.  
    # - The "out_id" argument will determine the label given to the combined channel data. 
    combine_channels!(obs, [:A1,:B1,:C1], out_id=channel)

    # the input data cubes are already in the sky frame, so we dont need to use the rotate_to_sky_axes! function
    
    # We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
    interpolate_nans!(obs.channels[channel])

    # Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
    # and replace the errors in the cube with these, since the provided errors are typically underestimated.
    # You can skip this step if you wish to use the default errors.
    calculate_statistical_errors!(obs.channels[channel])
    
    # Save the pre-processed data as a FITS file so it can be quickly reloaded later
    save_fits(".", obs, channel);
end

[ Info: Aligning World Coordinate Systems for channels [:A1, :B1, :C1]...
[ Info: The centroid offset relative to channel A1 for channel A1 is [0.0, 0.0]


[ Info: The centroid offset relative to channel A1 for channel B1 is [6.353058523700383e-9, 1.9300436768787677e-6]
[ Info: The centroid offset relative to channel A1 for channel C1 is [-7.250885580845079e-7, -1.2127456905375311e-6]


[ Info: Reprojecting NGC 7469 channel A1 onto the optimal (47, 43) WCS grid...
[ Info: Reprojecting NGC 7469 channel B1 onto the optimal (47, 43) WCS grid...


Progress:   0%|▏                                        |  ETA: 0:01:30

Progress:   1%|▍                                        |  ETA: 0:01:21

...

Progress:  99%|████████████████████████████████████████▊|  ETA: 0:00:01

Progress: 100%|████████████████████████████████████████▊|  ETA: 0:00:00

Progress: 100%|████████████████████████████████████████▉|  ETA: 0:00:00

Progress: 100%|█████████████████████████████████████████| Time: 0:01:36


[ Info: Writing FITS file from Observation object


We next create an aperture to define the region of interest that we would like to fit. We can do this with the `make_aperture` function. We can customize the aperture's shape, centroid, radius, etc.

In [6]:
# - The first argument is the data cube
# - The second argument is the aperture shape, which may be one of: (Circular, Rectangular, Elliptical)
# - Next are the right ascension in sexagesimal hours and the declination in sexagesimal degrees
# - The next arguments depend on the aperture shape:
#    - For circles, it is the radius in arcseconds
#    - For rectangles, it is the width in arcseconds, height in arcseconds, and rotation angle in degrees
#    - For ellipses, it is the semimajor axis in arcseconds, semiminor axis in arcseconds, and rotation angle in degrees
# - The auto_centroid argument, if true, will adjust the aperture centroid to the closest peak in brightness
# - The scale_psf argument, if true, will create a series of apertures with increasing radii that scale at the same rate as the PSF
ap = make_aperture(obs.channels[channel], :Circular, "23:03:15.610", "+8:52:26.10", 0.5, auto_centroid=true)

[ Info: Creating a circular aperture at 23:03:15.610, +8:52:26.10
[ Info: Aperture centroid adjusted to -0h56m44.385137045351115s, 8d52m26.130627601976215s


8×9 Photometry.Aperture.CircularAperture{Float64} with indices 20:27×19:27:
 0.0        0.0       0.180113  …  0.473967  0.023267    0.0
 0.0        0.216748  0.961375     1.0       0.706671    0.00607318
 0.0        0.783618  1.0          1.0       1.0         0.318239
 0.0451802  0.996812  1.0          1.0       1.0         0.576613
 0.0364361  0.99086   1.0          1.0       1.0         0.561917
 0.0        0.735994  1.0       …  1.0       1.0         0.270615
 0.0        0.159232  0.921269     1.0       0.614865    0.000257369
 0.0        0.0       0.109126     0.362874  0.00398031  0.0

Finally, we create the `CubeFitter` object and call the `fit_cube!` function to fit the data. To ensure we fit the data within the aperture, we must provide the `ap` argument.

If you instead wish to fit each spaxel individually, you may omit the `ap` argument. Be warned that this will take substantially longer. If you wish to fit each spaxel individually, it is recommended to enable the "parallel" option to allow multiple spaxels to be fit simultaneously. Doing this will also allow the code to produce 2D parameter maps of each fit parameter.

In [7]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.
cube_fitter = CubeFitter(obs.channels[channel], obs.z, run_name; parallel=false, plot_spaxels=:both, 
    plot_maps=true, save_fits=true, use_pah_templates=false, fit_sil_emission=true, save_full_model=false)

# Call the fit_cube! function on the cube_fitter object, using the aperture we defined.
fit_cube!(cube_fitter, ap)

[ Info: Preparing output directories
┌ Info: 
│ 
│ #############################################################################
│ ######## BEGINNING FULL CUBE FITTING ROUTINE FOR NGC_7469_ch1_nuc_aperture_mir ########
│ #############################################################################
│ 
│ ------------------------
│ Worker Processes:     4
│ Threads per process:  1
└ ------------------------


[ Info: ===> Preparing output data structures... <===
[ Info: Performing aperture photometry to get an integrated spectrum...


[ Info: ===> Beginning integrated spectrum fitting... <===
[ Info: ===> Generating parameter maps and model cubes... <===


[ Info: ===> Writing FITS outputs... <===
┌ Info: 
│ 
│ #############################################################################
│ ################################### Done!! ##################################
└ #############################################################################


(CubeFitter{Float64, Int64, ComplexF64}(DataCube([4.82172402445037, 4.822511180406426, 4.823298336362481, 4.824085492318536, 4.824872648274592, 4.825659804230647, 4.826446960186702, 4.827234116142758, 4.828021272098813, 4.828808428054868  …  7.519701216949793, 7.520488372905848, 7.521275528861904, 7.522062684817959, 7.522849840774014, 7.52363699673007, 7.524424152686125, 7.52521130864218, 7.525998464598236, 7.526785620554291], [NaN NaN … NaN NaN; NaN NaN … NaN NaN; … ; NaN NaN … NaN NaN; NaN NaN … NaN NaN;;; NaN NaN … NaN NaN; NaN NaN … NaN NaN; … ; NaN NaN … NaN NaN; NaN NaN … NaN NaN;;; NaN NaN … NaN NaN; NaN NaN … NaN NaN; … ; NaN NaN … NaN NaN; NaN NaN … NaN NaN;;; … ;;; NaN NaN … NaN NaN; NaN NaN … NaN NaN; … ; NaN NaN … NaN NaN; NaN NaN … NaN NaN;;; NaN NaN … NaN NaN; NaN NaN … NaN NaN; … ; NaN NaN … NaN NaN; NaN NaN … NaN NaN;;; NaN NaN … NaN NaN; NaN NaN … NaN NaN; … ; NaN NaN … NaN NaN; NaN NaN … NaN NaN], [NaN NaN … NaN NaN; NaN NaN … NaN NaN; … ; NaN NaN … NaN NaN; NaN NaN …

And the results can be found in the "output_[run_name]" directory!
Here is our fit of channel 1 of the nuclear spectrum of NGC 7469\*:

![results_1D](./NGC_7469_spaxel_1_1.png)

\*side note: it is generally not a good idea to fit only one channel here, this was done just as a quick example to get you started. The continuum components can be degenerate with the very flat extinction profile if you only fit one channel. LOKI has been built with the assumption/hope that one is fitting multiple MIRI channels combined into a single spectrum. Channel 2 in particular is crucial because it contains a large silicate absorption feature, but fitting just channel 2 by itself is also not advised because the shape of the extinction profile can be degenerate with the large wings of the PAH features to the left and right.